In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import mglearn

C:\Users\Jinny Pak\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\externals\six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

C:\Users\Jinny Pak\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [2]:
def mse_AIFrenz(y_true, y_pred):
    '''
    y_true: 실제 값
    y_pred: 예측 값
    '''
    diff = abs(y_true - y_pred)

    less_then_one = np.where(diff < 1, 0, diff)

    # multi-column일 경우에도 계산 할 수 있도록 np.average를 한번 더 씌움
    score = np.average(np.average(less_then_one ** 2, axis = 0))

    return score

In [3]:
train_file = './0_DataSet/train.csv'
test_file = './0_DataSet/test.csv'

In [4]:
sample_sub=pd.read_csv('sample_submission.csv')
test=pd.read_csv(test_file)
train=pd.read_csv(train_file)

In [5]:
X_train1=train.iloc[:4320,1:41]
X_train2=train.iloc[4320:,1:41]
y_17=train.iloc[:4320,41:-1]
y_18=train.iloc[4320:,59:]

## base-line 정의

In [10]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import VotingRegressor
models=[
    ('dt',DecisionTreeRegressor()), 
    ('rf',RandomForestRegressor()),
    ('xgb',XGBRegressor()),
    ('ab',AdaBoostRegressor()),
    ('br',BaggingRegressor()),
    ('gb',GradientBoostingRegressor()),
    ('sv',SVR()),
    ('lgbm',LGBMRegressor()),
    ('kn', KNeighborsRegressor()),
    ('ridge',Ridge()),
    ('lasso',Lasso())
    ]

In [24]:
# X-> Y00~17 학습 모델
vote  = VotingRegressor(models)
multi_model=MultiOutputRegressor(vote)
multi_model.fit(X_train1,y_17)

MultiOutputRegressor(estimator=VotingRegressor(estimators=[('dt',
                                                            DecisionTreeRegressor(ccp_alpha=0.0,
                                                                                  criterion='mse',
                                                                                  max_depth=None,
                                                                                  max_features=None,
                                                                                  max_leaf_nodes=None,
                                                                                  min_impurity_decrease=0.0,
                                                                                  min_impurity_split=None,
                                                                                  min_samples_leaf=1,
                                                                                  min_samples_split=2,
               

In [ ]:
# 3일간의 Y00~17 예측값
pred_y17=multi_model.predict(X_train2)
pred_y17_df=pd.DataFrame(pred_y17)

In [ ]:
# Y00~17 3일간의 예측값 -> Y18 3일 학습
vote2  = VotingRegressor(models)
multi_model2=MultiOutputRegressor(vote2)
multi_model2.fit(pred_y17,y_18)

## 제출할 예측값

In [27]:
# X_test -> Y00~17 예측
pred_test=multi_model.predict(test.iloc[:,1:])
pred_test_df=pd.DataFrame(pred_test)

# Y00~17 -> Y18 예측
pred_test2=multi_model2.predict(pred_test_df)
pred_test_df2=pd.DataFrame(pred_test2)
pred_test_df2

,0
0,19.960984
1,20.270790
2,19.988945
3,19.886526
4,19.643114
...,...
11515,26.538323
11516,26.889662
11517,26.821821
11518,26.285530
